In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Соберем названия книг и ссылки на них

In [ ]:
page = 'https://ru.wikipedia.org/w/index.php?title=Категория:Литературные_произведения_по_алфавиту&from=0' # стартовая страница
names = []
urls = []

In [ ]:
def read_pages(page, names, urls):
  while page:
    print('Current page:', page)

    response = requests.get(page)
    if (response.status_code != 200):
      print('Bad response!')

    soup = BeautifulSoup(response.content, 'html.parser')

    books_code = soup.find_all('div', 'mw-category-group')
    
    if not books_code:
      print('Books_code was not read!')

    book_lis = []
    for book_code in books_code:
      book_lis.extend(book_code.find_all('li'))
    
    if not book_lis:
      print('Book_lis was not read!')

    # выделим ссылки на книги
    book_urls = ['https://ru.wikipedia.org' + li.a['href'] for li in book_lis]

    # выделим названия книг
    book_names = [li.text for li in book_lis]

    names.extend(book_names)
    urls.extend(book_urls)

    #выделим ссылку на следующую страницу
    next_page_code = soup.find_all('a', text='Следующая страница')
    if next_page_code:
      page = 'https://ru.wikipedia.org' + next_page_code[0]['href']
    else:
      page = ''

In [ ]:
read_pages(page, names, urls)

Current page: https://ru.wikipedia.org/w/index.php?title=Категория:Литературные_произведения_по_алфавиту&from=0
Current page: https://ru.wikipedia.org/w/index.php?title=%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9B%D0%B8%D1%82%D0%B5%D1%80%D0%B0%D1%82%D1%83%D1%80%D0%BD%D1%8B%D0%B5_%D0%BF%D1%80%D0%BE%D0%B8%D0%B7%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83&pagefrom=%D0%90%D0%BD%D0%BD%D0%B0+%D0%9A%D0%B0%D1%80%D0%B5%D0%BD%D0%B8%D0%BD%D0%B0&subcatfrom=0&filefrom=0#mw-pages
Current page: https://ru.wikipedia.org/w/index.php?title=%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9B%D0%B8%D1%82%D0%B5%D1%80%D0%B0%D1%82%D1%83%D1%80%D0%BD%D1%8B%D0%B5_%D0%BF%D1%80%D0%BE%D0%B8%D0%B7%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83&pagefrom=%D0%91%D0%B5%D0%B4%D0%BD%D1%8B%D0%B5-%D0%BD%D0%B5%D1%81%D1%87%D0%B0%D1%81%D1%82%D0%BD%D1%8B%D0%B5+%28%D1%80%D0%BE%D0%BC%D0%

In [ ]:
len(names) # посмотрим количество собранных книг

7200

Соберем авторов, жанры и картинки при их наличии

In [ ]:
authors = []
genres = []
pictures = []

In [ ]:
def read_author(table):
  check_author = table[0].find_all('th', text='Автор') # проверим, что автор указан
  if not check_author:
    return None
  author = table[0].find_all('th', text='Автор')[0].parent.find_all('a') # имя автора со ссылкой
  if not author:
    author = table[0].find_all('th', text='Автор')[0].parent.find_all('td')[0].text.strip() # имя автора без ссылки
  else:
    author = author[0].text.strip()
  return author


def read_genre(table):
  genre_code = table[0].find_all('th', text='Жанр') # проверяем, что есть жанр
  if not genre_code:
    genre_code = table[0].find_all('th', text='Жанр / тематика') # проверяем, может есть жанр / тематика
  if not genre_code:
    return None

  genre_list = genre_code[0].parent.find_all('a') # если жанр с ссылкой
  if not genre_list:
    genre_list = genre_code[0].parent.find_all('td') # если жанр без ссылки
    
  genre = [elem.text.strip() for elem in genre_list]

  return genre


def read_pic(table):
  check_pic = table[0].find_all('img')
  if not check_pic:
    return None
  return check_pic[0]['src']

In [ ]:
i = 0 # счетчик, чтобы выдеть сколько книг уже обработано

for url in urls:
  print('Current url:', url, 'i =', i)
  i += 1

  response = requests.get(url)
  if response.status_code != 200:
    print('Bad status code!')

  soup = BeautifulSoup(response.content, 'html.parser')
  table = soup.find_all('table', 'infobox')

  if not table:
    authors.append(None)
    genres.append(None)
    pictures.append(None)
    continue

  authors.append(read_author(table))
  genres.append(read_genre(table))
  pictures.append(read_pic(table))

Выходные данные были обрезаны до нескольких последних строк (5000).
Current url: https://ru.wikipedia.org/wiki/%D0%98%D0%BF%D0%BF%D0%BE%D0%BB%D0%B8%D1%82_(%D0%95%D0%B2%D1%80%D0%B8%D0%BF%D0%B8%D0%B4) i = 2200
Current url: https://ru.wikipedia.org/wiki/%D0%98%D1%81%D0%B0%D0%B0%D0%BA_%D0%9B%D0%B0%D0%BA%D0%B5%D0%B4%D0%B5%D0%BC_(%D1%80%D0%BE%D0%BC%D0%B0%D0%BD) i = 2201
Current url: https://ru.wikipedia.org/wiki/%D0%98%D1%81%D0%BA%D0%B0%D0%BD%D0%B4%D0%B5%D1%80-%D0%BD%D0%B0%D0%BC%D0%B5 i = 2202
Current url: https://ru.wikipedia.org/wiki/%D0%98%D1%81%D0%BA%D0%B0%D1%82%D0%B5%D0%BB%D0%B8_%D0%BD%D0%B5%D0%B1%D0%B0 i = 2203
Current url: https://ru.wikipedia.org/wiki/%D0%98%D1%81%D0%BA%D0%B0%D1%82%D0%B5%D0%BB%D1%8C_(%D1%80%D0%BE%D0%BC%D0%B0%D0%BD) i = 2204
Current url: https://ru.wikipedia.org/wiki/%D0%98%D1%81%D0%BA%D0%B0%D1%82%D0%B5%D0%BB%D1%8C_%D0%BF%D1%80%D0%B8%D0%BA%D0%BB%D1%8E%D1%87%D0%B5%D0%BD%D0%B8%D0%B9_(%D1%80%D0%B0%D1%81%D1%81%D0%BA%D0%B0%D0%B7) i = 2205
Current url: https://ru.wikipedia.

In [ ]:
len(names), len(urls), len(authors), len(genres), len(pictures)

(7200, 7200, 7200, 7200, 7200)

Внесем все данные в таблицу

In [ ]:
df = pd.DataFrame(
    {
        'Название': names,
        'Автор': authors,
        'Жанр': genres,
        'Картинка': pictures,
        'Ссылка': urls,
    }
)

In [ ]:
df

,Название,Автор,Жанр,Картинка,Ссылка
0,1Q84,Мураками Харуки,[роман],//upload.wikimedia.org/wikipedia/commons/thumb...,https://ru.wikipedia.org/wiki/1Q84
1,2BRO2B,Курт Воннегут,[рассказ],None,https://ru.wikipedia.org/wiki/2BRO2B
2,11/22/63,Стивен Кинг,"[альтернативная история, фантастика]",//upload.wikimedia.org/wikipedia/ru/a/a8/22_no...,https://ru.wikipedia.org/wiki/11/22/63
3,13½ жизней капитана по имени Синий Медведь,Вальтер Мёрс,[фэнтези],//upload.wikimedia.org/wikipedia/ru/thumb/6/6a...,https://ru.wikipedia.org/wiki/13%C2%BD_%D0%B6%...
4,14 декабря (роман),Д. С. Мережковский,[исторический роман],None,https://ru.wikipedia.org/wiki/14_%D0%B4%D0%B5%...
...,...,...,...,...,...
7195,Yoshinaga-san Chi no Gargoyle,Сэннэндо Тагути,"[Жанр, сёнэн]",//upload.wikimedia.org/wikipedia/ru/thumb/c/c7...,https://ru.wikipedia.org/wiki/Yoshinaga-san_Ch...
7196,Yuki Yuna is a Hero,Такахиро,"[Жанр, сэйнэн]",//upload.wikimedia.org/wikipedia/ru/thumb/9/93...,https://ru.wikipedia.org/wiki/Yuki_Yuna_is_a_Hero
7197,Yume Nikki,None,"[Жанр, квест, психологические ужасы]",//upload.wikimedia.org/wikipedia/ru/thumb/b/b4...,https://ru.wikipedia.org/wiki/Yume_Nikki
7198,Yuri Kuma Arashi,Кунихико Икухара,"[фантастика, юри]",//upload.wikimedia.org/wikipedia/ru/thumb/d/da...,https://ru.wikipedia.org/wiki/Yuri_Kuma_Arashi


In [ ]:
df.to_csv('books.csv', encoding='utf-8', index=False)

In [ ]:
df.to_excel('books.xlsx', encoding='utf-8', index=False)